This seminar will help you to do your homework on creating a panorama view. Let's install some useful libraries

In [ ]:
# !unzip input.zip

In [ ]:
%pip install opencv-python matplotlib numpy ipython-autotime

In [ ]:
import glob
import itertools
import math
import os.path as osp
import time

import cv2
import matplotlib
import numpy as np

# matplotlib.use('TkAgg')  # Uncomment for macOS

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (20.0, 16.0)
plt.rcParams['image.interpolation'] = 'bilinear'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2
%load_ext autotime

# Panorama stitching


![](https://media.springernature.com/lw685/springer-static/image/prt%3A978-0-387-31439-6%2F9/MediaObjects/978-0-387-31439-6_9_Part_Fig1-13_HTML.jpg)

The seminar consists of two parts. The first part is dedicated to panorama stitching, and the second part covers camera calibration. Camera calibration is not mandatory, but there is no camera without distortion, as a result of which straight lines will not appear straight. This is particularly evident in wide-angle cameras, so calibration is essential in such cases.



Example of distortion create by [demo](https://jywarren.github.io/fisheyegl/example/#a=3.276&b=2.223&Fx=2&Fy=0.26&scale=1.243&x=0&y=0)



## Load Images

Loading images. Here's an example of three photos taken at home that we want to stitch into a panorama. In your homework, you have five, so you'll have to put in some work :)

In [ ]:
input_dir = 'input'
ipaths = sorted(glob.glob(osp.join(input_dir, 'img_*.jpg')))
n_imgs = len(ipaths)

images_info = dict()
for i_num, ipath in enumerate(ipaths):
    img = plt.imread(ipath)
    images_info[ipath] = {'img': img}
    plt.subplot(1, n_imgs, i_num + 1)
    plt.title(osp.basename(ipath), fontsize=20)
    plt.axis('off')
    plt.imshow(images_info[ipath]['img'])
plt.show()

## Keypoints detection + descriptors

To stitch the panorama, we need to obtain the homography matrix. We can obtain the homography matrix using the RANSAC method. For generating keypoints, we will use SIFT.

It's worth noting that SIFT is performed on grayscale images. This is done to speed up the process, as SIFT on RGB does not significantly improve the quality.

By varying the SIFT parameters, you can obtain a different number of keypoints and types. Therefore, try experimenting with these parameters.

![](https://www.codeproject.com/KB/recipes/619039/SIFT.JPG)

In [ ]:
def keypoints_detection_sift(input_img):
    gray = cv2.cvtColor(input_img, cv2.COLOR_RGB2GRAY)
    sift = cv2.SIFT_create()
    kps, dscrs = sift.detectAndCompute(gray, None)
    return kps, dscrs

Here, we can visualize keypoint descriptors from SIFT and can visualize them.

In [ ]:
for i_num, ipath in enumerate(images_info):
    img = images_info[ipath]['img']
    keypoints, descriptors = keypoints_detection_sift(img)
    images_info[ipath]['keypoints'] = keypoints
    images_info[ipath]['descriptors'] = descriptors
    plt.subplot(1, n_imgs, i_num + 1)
    plt.title(osp.basename(ipaths[i_num]), fontsize=20)
    plt.imshow(cv2.drawKeypoints(
        img, keypoints, None,
        flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
    ))
    plt.axis('off')
plt.show()

There are plenty of keypoints in the image, and it can be noticed that they come in different sizes, meaning they come from different octaves. It's also noticeable that there are too many of them, and we would like to filter them out :)

**HINT:** It's very pleasant to detect keypoints in the text; they will turn out to be corners that match well with each other. Sort the keypoints by size.

*Off-topic:* We can speed up the calculation using int8 cast, but it's not essential for our task.

In [ ]:
keypoints, descriptors = images_info[ipaths[0]]['keypoints'], \
    images_info[ipaths[0]]['descriptors']
keypoint = sorted(keypoints, key=lambda x: x.size, reverse=True)[0]
for field in dir(keypoint):
    if not field.startswith('_'):
        print(f'{field:10} {getattr(keypoint, field)}')

In [ ]:
_ = plt.figure(figsize=(10, 8))
plt.imshow(cv2.drawKeypoints(
    images_info[ipaths[0]]['img'], [keypoint, ], None,
    flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
)
plt.axis('off')
plt.show()

Let's look how good panorama should looks like. (Not good 😞)

In [ ]:
%pip install kornia kornia_rs # kornia_rs for Linux

In [ ]:
import kornia as K
from kornia.core import Tensor
from kornia.contrib import ImageStitcher
import kornia.feature as KF
import torch

def inference(file_1, file_2):
    img_1: Tensor = K.io.load_image(file_1, K.io.ImageLoadType.RGB32)
    img_1 = img_1[None]  # 1xCxHxW / fp32 / [0, 1]
    img_2: Tensor = K.io.load_image(file_2, K.io.ImageLoadType.RGB32)
    img_2 = img_2[None]  # 1xCxHxW / fp32 / [0, 1]

    IS = ImageStitcher(KF.LoFTR(pretrained='indoor'), estimator='ransac')
    with torch.no_grad():
        result = IS(img_1, img_2)

    return K.tensor_to_image(result[0])

In [ ]:
panorama = inference('/content/input/img_00.jpg', '/content/input/img_01.jpg')

In [ ]:
plt.imshow(panorama)

Example of panorama with Kornia... Not good

## Keypoints matching

### Bruteforce + Cross-check

For matching, we will use a simple brute-force algorithm. We will calculate the Euclidean distance and for each keypoint from the first image, we will find the corresponding keypoint on the second image.

Additionally, we can implement cross-checking. If we run the keypoint matching procedure from the second image to the first, not all points will be mapped to the exact same points. Therefore, we can check the reliability of some keypoints.

In [ ]:
def keypoints_matching_cross_check(dscrs1, dscrs2):
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
    cross_matches = bf.match(dscrs1, dscrs2)
    return cross_matches

In [ ]:
for i_num, ipath in enumerate(images_info):
    print(images_info[ipath]['descriptors'])

In [ ]:
matches = keypoints_matching_cross_check(
    images_info[ipaths[0]]['descriptors'],
    images_info[ipaths[1]]['descriptors']
)

Now let's see how the points have been mapped to each other.

In [ ]:
img_matches = cv2.drawMatches(
    images_info[ipaths[0]]['img'],
    images_info[ipaths[0]]['keypoints'],
    images_info[ipaths[1]]['img'],
    images_info[ipaths[1]]['keypoints'],
    matches,
    None,  # output image
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
)
plt.imshow(img_matches)
plt.axis('off')
plt.show()

There are a couple of key points on the straight lines, and we don't like them because the differences between the points on the line are not very significant.

There are also a huge number of points on the keyboard keys. Since the keys are very similar to each other, it will be difficult to organize a correct matching on them. Therefore, we will filter them using the Ratio test.

### KNN + Ratio test

By reducing the number of points based on distance, we ensure that there are no points in the vicinity that have similar points. You can play around with this parameter, decreasing or increasing it as per your requirement.

In [ ]:
def keypoints_matching_knn(dscrs1, dscrs2):
    bf = cv2.BFMatcher()
    tic = time.time_ns()
    knn_matches = bf.knnMatch(dscrs1, dscrs2, k=2)
    toc = time.time_ns()
    print(f'Matching time: {(toc - tic) / 10e6 :.5f} ms')
    good_matches = []
    for neighbour_1, neighbour_2 in knn_matches:
        if neighbour_1.distance < 0.75 * neighbour_2.distance:
            good_matches.append(neighbour_1)
    return good_matches

In [ ]:
matches = keypoints_matching_knn(
    images_info[ipaths[0]]['descriptors'],
    images_info[ipaths[1]]['descriptors']
)

In [ ]:
img_matches = cv2.drawMatches(
    images_info[ipaths[0]]['img'],
    images_info[ipaths[0]]['keypoints'],
    images_info[ipaths[1]]['img'],
    images_info[ipaths[1]]['keypoints'],
    matches,
    None,  # output image
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
)
plt.imshow(img_matches)
plt.axis('off')
plt.show()

### FLANN

The search for matches in our image is quite fast, but it's still worth mentioning the methods for fast nearest neighbor search. For example, when we are performing face matching with a database of 150 million faces, using the classical nearest neighbor search is not advantageous. Therefore, FLANN, which is based on KD-trees, can be applied.

In [ ]:
def keypoints_matching_flann(dscrs1, dscrs2):
    index_params = dict(
        algorithm=1,  # FLANN_INDEX_KDTREE
        trees=5
    )
    search_params = dict(checks=20)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    tic = time.time_ns()
    flann_matches = flann.knnMatch(dscrs1, dscrs2, k=2)
    toc = time.time_ns()
    print(f'Matching time: {(toc - tic) / 10e6 :.5f} ms')
    good_matches = []
    for neighbour_1, neighbour_2 in flann_matches:
        if neighbour_1.distance < 0.75 * neighbour_2.distance:
            good_matches.append(neighbour_1)
    return good_matches

In [ ]:
matches = keypoints_matching_flann(
    images_info[ipaths[0]]['descriptors'],
    images_info[ipaths[1]]['descriptors']
)

In [ ]:
img_matches = cv2.drawMatches(
    images_info[ipaths[0]]['img'],
    images_info[ipaths[0]]['keypoints'],
    images_info[ipaths[1]]['img'],
    images_info[ipaths[1]]['keypoints'],
    matches,
    None,  # output image
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
)
plt.imshow(img_matches)
plt.axis('off')
plt.show()

If we use our images, then the time difference is small. To let you feel how big the difference is, let's increase the number of points to see how much the matching differs in efficiency.

It's not very fair for the KD tree because the points will be duplicated in the image, but it helps to feel the difference excellently. Overall, KNN is not that slow, so it is not recommended to use FLANN, as there are few images.

In [ ]:
print(f'Descriptors shape on the image_0: {images_info[ipaths[0]]["descriptors"].shape}')
print(f'Descriptors shape on the image_1: {images_info[ipaths[1]]["descriptors"].shape}')
mock_descriptors_1 = np.repeat(images_info[ipaths[0]]['descriptors'], 100, axis=0)
mock_descriptors_2 = np.repeat(images_info[ipaths[1]]['descriptors'], 100, axis=0)
print(f'Descriptors shape on the mock_image_0: {mock_descriptors_1.shape}')
print(f'Descriptors shape on the mock_image_1: {mock_descriptors_2.shape}')

In [ ]:
_ = keypoints_matching_knn(mock_descriptors_1, mock_descriptors_2)

In [ ]:
_ = keypoints_matching_flann(mock_descriptors_1, mock_descriptors_2)

### Find matches for images

In [ ]:
matches_images = dict()
for ipath_0, ipath_1 in itertools.permutations(ipaths, 2):
    matches_pair = keypoints_matching_knn(
        images_info[ipath_0]['descriptors'],
        images_info[ipath_1]['descriptors']
    )
    matches_images[(ipath_0, ipath_1)] = matches_pair

## Homography

To build a homography matrix, we will use the RANSAC method. RANSAC handles outliers well. (You need 4 points for homography, you can take more through SVD).

One of the important parameters you can play with is the threshold.

In [ ]:
homographies = dict()
for (ipath_0, ipath_1), matches in matches_images.items():
    src_pts = np.float32(
        [images_info[ipath_0]['keypoints'][m.queryIdx].pt for m in matches]
    ).reshape(-1, 1, 2)
    dst_pts = np.float32(
        [images_info[ipath_1]['keypoints'][m.trainIdx].pt for m in matches]
    ).reshape(-1, 1, 2)
    M, mask = cv2.findHomography(
        src_pts,
        dst_pts,
        cv2.RANSAC,
        ransacReprojThreshold=.5,
        maxIters=20000,
        confidence=0.995
    )
    homographies[ipath_0, ipath_1] = (M, mask.ravel().tolist())

In [ ]:
M

When we have constructed the homography matrix, we can observe how one image is projected onto another.  

In [ ]:
for ipath_0, ipath_1 in itertools.permutations(ipaths, 2):
    matches = matches_images[ipath_0, ipath_1]
    M, matches_mask = homographies[ipath_0, ipath_1]
    img_0, img_1_polylines = images_info[ipath_0]['img'].copy(), \
        images_info[ipath_1]['img'].copy()
    rows_0, cols_0 = img_0.shape[:2]
    rows_1, cols_1 = img_1_polylines.shape[:2]
    pts = np.float32(
        [[0, 0], [0, rows_0], [cols_0, rows_0], [cols_0, 0]]).reshape(-1, 1, 2)
    dst = cv2.perspectiveTransform(pts, M)
    img_1_polylines = cv2.polylines(
        img_1_polylines, [np.int32(dst)], True, 255, 3, cv2.LINE_AA
    )
    img_matches = cv2.drawMatches(
        img_0,
        images_info[ipath_0]['keypoints'],
        img_1_polylines,
        images_info[ipath_1]['keypoints'],
        matches,
        None,  # output image
        matchesMask=matches_mask,
        flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
    )
    plt.title(
        f'Pair of images: {ipath_0} and {ipath_1} ({sum(matches_mask)} matches)',
        fontsize=20
    )
    plt.imshow(img_matches)
    plt.axis('off')
    plt.show()

## Stitching images

In [ ]:
M, _ = homographies[ipaths[1], ipaths[0]]
img_0, img_1 = images_info[ipaths[0]]['img'], images_info[ipaths[1]]['img']
img_0_1 = cv2.warpPerspective(
    img_1, M, (img_0.shape[1] + img_1.shape[1], img_1.shape[0])
)
img_0_1[0:img_0.shape[0], 0:img_0.shape[1]] = img_0

# Some postprocessing
img_0_1_mask = np.where(img_0_1.sum(axis=2) != 0, 1, 0).astype('uint8')
kernel = np.ones((5, 5), np.uint8)
img_0_1_mask = cv2.morphologyEx(img_0_1_mask, cv2.MORPH_CLOSE, kernel)
img_0_1_mask = cv2.erode(img_0_1_mask, kernel, iterations=1)
img_0_1_mask = np.dstack([img_0_1_mask, ] * 3)

In [ ]:
plt.imshow(img_0_1)
plt.axis('off')
plt.show()

In [ ]:
M, _ = homographies[ipaths[2], ipaths[0]]
img_2 = images_info[ipaths[2]]['img']
img_2_pano = cv2.warpPerspective(
    img_2, M, (img_0.shape[1] + img_1.shape[1], img_1.shape[0])
)
panorama = np.where(img_0_1_mask != 0, img_0_1, img_2_pano)

In [ ]:
plt.imshow(panorama)
plt.show()

Here, the first image is being stitched with the second one, so the quality is slightly worse, but you can stitch the first image with the second one and the quality will increase.

### Hints

To achieve good quality, try varying the parameters mentioned above. Iterate through this process to obtain excellent images.

Additionally, you can implement a more sophisticated image blending by normalizing the image colors or blending with consideration of the alpha channel.

This will help you address issues arising from the warp perspective.


# Camera calibration

To better understand the camera parameters one can try these demos

Extrinsic parameters demo:

https://ksimek.github.io/2012/08/22/extrinsic/#:~:text=%3D%20%2DRC.-,Try%20it%20out!,-Below%20is%20an

Intrinsic parameters demo:

https://ksimek.github.io/2013/08/13/intrinsic/#:~:text=in%20image%20space.-,Demo,-The%20demo%20below


To better understand distortion, check out this.

https://www.tangramvision.com/blog/camera-modeling-exploring-distortion-and-distortion-models-part-i


For camera calibration, we will use a special pattern called a chessboard. Its parameters are well known to us, so calibrating the camera with its help will be easier than through the pipeline we used for panorama construction. The image of the chessboard can be found in the OpenCV distribution on [GitHub](https://github.com/opencv/opencv/blob/4.x/samples/data/chessboard.png). It needs to be printed and photographed from different angles and distances to obtain an adequate number of images for camera calibration. The total number of images should be no less than 10.

In [ ]:
input_dir = 'input'
chessboard_ipaths = sorted(glob.glob(osp.join(input_dir, 'chessboard_*.jpg')))
n_chessboard_imgs = len(chessboard_ipaths)
print(f'Number of chessboard images: {n_chessboard_imgs}')

On each image, it is necessary to find the corners of the chessboard. For this purpose, the `cv2.findChessboardCorners` function is used. This function provides correspondence between the corners of the chessboard in the real world (object points) and the points on the image (image points).

After finding the corners, their location can be refined using the `cv2.cornerSubPix` function. You can use this function if you find the calibration quality obtained without it to be insufficient.

In [ ]:
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(8,5,0)
objp = np.zeros((6 * 9, 3), np.float32)
objp[:, :2] = np.mgrid[0:9, 0:6].T.reshape(-1, 2)
# Arrays to store object points and image points from all the images
obj_points = []  # 3d points in real world space
img_points = []  # 2d points in image plane.

In [ ]:
_ = plt.figure(figsize=(16, 36))
for i_num, ipath in enumerate(chessboard_ipaths):
    img = cv2.cvtColor(cv2.imread(ipath), cv2.COLOR_BGR2RGB)
    img_grayscale = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    result, corners = cv2.findChessboardCorners(img_grayscale, (9, 6), None)
    if result:
        obj_points.append(objp)
        img_points.append(corners)

        cv2.drawChessboardCorners(img, (9, 6), corners, result)
        plt.subplot(math.ceil(n_chessboard_imgs / 3), 3, i_num + 1)
        plt.title(osp.basename(ipath), fontsize=12)
        plt.axis('off')
        plt.imshow(img)
plt.show()

Having correspondences between the corners of the chessboard in the real world and the points on the image, you can calibrate the camera. For this purpose, the `cv2.calibrateCamera` function is used. As a result of calibration, we will obtain the camera matrix and distortion coefficients.

In [ ]:
img = cv2.cvtColor(cv2.imread(chessboard_ipaths[-1]), cv2.COLOR_BGR2RGB)
img_grayscale = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
result, mtx, dist, r_vecs, t_vecs = cv2.calibrateCamera(
    objectPoints=obj_points,
    imagePoints=img_points,
    imageSize=img_grayscale.shape[::-1],
    cameraMatrix=None,
    distCoeffs=None
)
print(f'Camera matrix:\n{mtx}')
print(f'Distortion coefficients:\n{dist}')

In [ ]:
np.savetxt('mtx.txt', mtx)
np.savetxt('dist.txt', dist)

These parameters can be used to correct images obtained with this camera. The `cv2.undistort` function is used for this purpose. Additionally, one can use the `cv2.getOptimalNewCameraMatrix` function to obtain a new camera matrix. This new matrix will contain only those pixels that carry information about the image.

In [ ]:
rows, cols = img.shape[:2]
new_camera_mtx, roi = cv2.getOptimalNewCameraMatrix(
    cameraMatrix=mtx,
    distCoeffs=dist,
    imageSize=(cols, rows),
    alpha=1,
    newImgSize=(cols, rows)
)
print(f'New camera matrix:\n{new_camera_mtx}')
print(f'Region of interest:\n{roi}')

In [ ]:
dst = cv2.undistort(
    src=img,
    cameraMatrix=mtx,
    distCoeffs=dist,
    dst=None,
    newCameraMatrix=new_camera_mtx
)

plt.imshow(dst)
plt.axis('off')
plt.show()

In [ ]:
roi_x, roi_y, roi_w, roi_h = roi
dst = dst[roi_y:roi_y + roi_h, roi_x:roi_x + roi_w]
plt.imshow(dst)
plt.axis('off')
plt.show()